# Modlling for engagement 

now we have the features and the lables, we are ready for modelling 
- Classification vs regression
- 9 vs 5 vs 3 scales lables 
- 2.5 vs 5 s window (as is features, average featuers and concatenate features)
- Two-stream Fusion on RGB + Flow 

This should be done over:
- different network artchictures
- cross different familys cross-validation 

Later on:
- Handcrafted features 

### imports and vars

In [1]:
import sys
import numpy as np
import os
import pandas as pd
import itertools
import pickle
import warnings
warnings.filterwarnings('ignore')

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

import logging, sys
logging.disable(sys.maxsize)

import smote_variants as sv
from collections import Counter
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from operator import itemgetter

lables_path = './labels/'
features_path = './features/'
results_path = './modelling_results/'

### Keras modeling class 

### help functions 


In [2]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

from hpbandster.core.worker import Worker
from hpbandster.optimizers import BOHB as BOHB
import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

import tensorflow_addons as tfa

import logging
logging.basicConfig(level=logging.ERROR)

class KerasWorker(Worker):
    def __init__(self, input_shape, output_shape, problemType,
                 x_train, y_train, x_validation, y_validation,
                 x_test, y_test, shared_directory, **kwargs):
            super().__init__(**kwargs)
            self.input_shape = (input_shape, )
            self.num_classes = output_shape
            self.batch_size = 64
            self.save_dic = shared_directory
            
            self.problemType = problemType

            self.x_train, self.y_train = x_train, y_train
            self.x_validation, self.y_validation = x_validation, y_validation
            self.x_test, self.y_test = x_test, y_test

    def compute(self, config, budget, working_directory, *args, **kwargs):
            model = Sequential()
            model.add(Dense(units=config['start_neurons_units'],
                            # activation=config['start_neurons_activation'],
                            activation='relu',
                            input_shape=self.input_shape))


            if config['num_dense_layers'] > 1:
                model.add(Dense(units=config['dense1_units'],
                                # activation=config['dense1_activation'],
                                activation='relu',
                                input_shape=self.input_shape))
                model.add(Dropout(config['dropout1_rate']))

            if config['num_dense_layers'] > 2:
                model.add(Dense(units=config['dense2_units'],
                                # activation=config['dense2_activation'],
                                activation='relu',
                                input_shape=self.input_shape))
                model.add(Dropout(config['dropout2_rate']))

            model.add(Dense(self.num_classes, activation='softmax'))


            if config['optimizer'] == 'Adam':
                    optimizer = tf.keras.optimizers.Adam(lr=config['lr'])
            else:
                    optimizer = tf.keras.optimizers.SGD(lr=config['lr'], momentum=config['sgd_momentum'])
            
            loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
            
            METRICS = [
                  tf.keras.metrics.TruePositives(name='tp'),
                  tf.keras.metrics.FalsePositives(name='fp'),
                  tf.keras.metrics.TrueNegatives(name='tn'),
                  tf.keras.metrics.FalseNegatives(name='fn'),
                  tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                  tf.keras.metrics.Precision(name='precision'),
                  tf.keras.metrics.Recall(name='recall'),
                  tf.keras.metrics.AUC(name='auc'),
                  tf.keras.metrics.BinaryCrossentropy(name='BinaryCrossentropy'),
                  tfa.metrics.MatthewsCorrelationCoefficient(num_classes=self.num_classes)
            ]

            #METRICS.append(tfa.metrics.MatthewsCorrelationCoefficient(num_classes=self.num_classes))
            
            the_columns = ['thisloss','tp','fp','tn','fn','acc','prec','rec','auc','BC','MCC']
            
            model.compile(
                loss=loss_fn,
                optimizer=optimizer,
                metrics=METRICS
            )

            # model.summary()
            _history = model.fit(self.x_train, self.y_train,
                              batch_size=self.batch_size,
                              epochs=int(budget),
                              verbose=0,
                              validation_data=(self.x_validation, self.y_validation))

            val_acc_per_epoch = _history.history['val_accuracy']
            best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

            model.fit(self.x_train, self.y_train,
                              batch_size=self.batch_size,
                              epochs=best_epoch,
                              verbose=0,
                              validation_data=(self.x_validation, self.y_validation))

            train_score = model.evaluate(self.x_train, self.y_train, verbose=0)
            val_score = model.evaluate(self.x_validation, self.y_validation, verbose=0)
            test_score = model.evaluate(self.x_test, self.y_test, verbose=0)
            
            #print(test_score)

            resultsDF = pd.DataFrame([train_score,val_score,test_score],
                                     columns=the_columns,
                                     index=["train_score", "val_score", "test_score"],)
            # print(resultsDF)
            test_predictions_baseline = model.predict(self.x_test)
            np.savetxt(os.path.join(self.save_dic,'testing_finalResults_true.out'), self.y_test, delimiter=',')
            np.savetxt(os.path.join(self.save_dic,'testing_finalResults_pred.out'), test_predictions_baseline, delimiter=',')

            return ({
                'loss': test_score[0],  
                'info':  resultsDF.to_dict('index')
            })
        
#             #import IPython; IPython.embed()
#             return ({
#                 'loss': 1-val_score[1], # remember: HpBandSter always minimizes!
#                 'info': {'test accuracy': test_score[1],
#                             'train accuracy': train_score[1],
#                             'validation accuracy': val_score[1],
#                             'number of parameters': model.count_params(),
#                         }

#             })


    @staticmethod
    def get_configspace():
            """
            It builds the configuration space with the needed hyperparameters.
            It is easily possible to implement different types of hyperparameters.
            Beside float-hyperparameters on a log scale, it is also able to handle categorical input parameter.
            :return: ConfigurationsSpace-Object
            """
            cs = CS.ConfigurationSpace()

            lr = CSH.UniformFloatHyperparameter('lr', lower=1e-6, upper=1e-1, default_value='1e-2', log=True)

            # For demonstration purposes, we add different optimizers as categorical hyperparameters.
            # To show how to use conditional hyperparameters with ConfigSpace, we'll add the optimizers 'Adam' and 'SGD'.
            # SGD has a different parameter 'momentum'.
            optimizer = CSH.CategoricalHyperparameter('optimizer', ['Adam', 'SGD'])

            sgd_momentum = CSH.UniformFloatHyperparameter('sgd_momentum', lower=0.0, upper=0.99, default_value=0.9, log=False)

            cs.add_hyperparameters([lr, optimizer, sgd_momentum])



            num_dense_layers =  CSH.UniformIntegerHyperparameter('num_dense_layers', lower=1, upper=3, default_value=2)

            start_neurons_units = CSH.UniformIntegerHyperparameter('start_neurons_units', lower=32, upper=512, default_value=32, log=True)
            dense1_units = CSH.UniformIntegerHyperparameter('dense1_units', lower=8, upper=128, default_value=16, log=True)
            dense2_units = CSH.UniformIntegerHyperparameter('dense2_units', lower=4, upper=64, default_value=8, log=True)

            cs.add_hyperparameters([num_dense_layers, start_neurons_units, dense1_units, dense2_units])

            # start_neurons_activation = CSH.CategoricalHyperparameter('start_neurons_activation', ['relu', 'tanh', 'sigmoid'])
            # dense1_activation = CSH.CategoricalHyperparameter('dense1_activation', ['relu', 'tanh', 'sigmoid'])
            # dense2_activation = CSH.CategoricalHyperparameter('dense2_activation', ['relu', 'tanh', 'sigmoid'])
            # start_neurons_activation = CSH.CategoricalHyperparameter('start_neurons_activation', ['relu'])
            # dense1_activation = CSH.CategoricalHyperparameter('dense1_activation', ['relu'])
            # dense2_activation = CSH.CategoricalHyperparameter('dense2_activation', ['relu'])
            #
            # cs.add_hyperparameters([start_neurons_activation, dense1_activation, dense2_activation])

            dropout1_rate = CSH.UniformFloatHyperparameter('dropout1_rate', lower=0.0, upper=0.9, default_value=0.5, log=False)
            dropout2_rate = CSH.UniformFloatHyperparameter('dropout2_rate', lower=0.0, upper=0.9, default_value=0.5, log=False)

            cs.add_hyperparameters([dropout1_rate, dropout2_rate])


            # The hyperparameter sgd_momentum will be used,if the configuration
            # contains 'SGD' as optimizer.
            cond = CS.EqualsCondition(sgd_momentum, optimizer, 'SGD')
            cs.add_condition(cond)

            # You can also use inequality conditions:
            cond = CS.GreaterThanCondition(dense1_units, num_dense_layers, 1)
            cs.add_condition(cond)

            cond = CS.GreaterThanCondition(dense2_units, num_dense_layers, 2)
            cs.add_condition(cond)

            return cs


In [3]:
def load_sub_dataset_twostream(familiesSet, label_folder, feature_folder):
    # append all rows of subjects, and their lables
    allFrames = np.array([])
    allLables = np.array([])
    
    rgb_feature_folder = feature_folder.replace(featureType,'rgb')
    flow_feature_folder = feature_folder.replace(featureType,'flow')
    for this_family in familiesSet:
        onlyfiles = [f for f in os.listdir(rgb_feature_folder) if
                       os.path.isfile(os.path.join(rgb_feature_folder, f))
                       and f.startswith(this_family + '_')]
        onlyfiles.sort()

        for this_file in onlyfiles:
            this_lable_file = this_file.replace('_'+this_file.split('_')[4],'')+'.npy'
            currLabel = np.load(os.path.join(label_folder,this_lable_file))
            
            rgb_currData = np.load(os.path.join(rgb_feature_folder,this_file))    
            flow_currData = np.load(os.path.join(flow_feature_folder,this_file.replace('rgb','flow')))
            
            currData = np.hstack((rgb_currData, flow_currData))
            
            if allFrames.shape[0] ==0:
                allFrames = currData
                allLables = currLabel
            else:
                allFrames = np.vstack((allFrames, currData))
                allLables = np.hstack((allLables, currLabel))

    return allFrames, allLables

def load_sub_dataset(familiesSet, label_folder, feature_folder, featureType):
    # append all rows of subjects, and their lables
    allFrames = np.array([])
    allLables = np.array([])
    
    if featureType == 'twostream':
        return load_sub_dataset_twostream(familiesSet, label_folder, feature_folder)
    
    for this_family in familiesSet:
        # F10_Interaction_1_P27_rgb.npy
        onlyfiles = [f for f in os.listdir(feature_folder) if
                       os.path.isfile(os.path.join(feature_folder, f))
                       and f.startswith(this_family + '_')]
        onlyfiles.sort()

        for this_file in onlyfiles:
            currData = np.load(os.path.join(feature_folder,this_file))            
            substr = '_'.join(this_file.split('_')[4:])
            this_lable_file = this_file.replace('_'+substr,'')+'.npy'
            currLabel = np.load(os.path.join(label_folder,this_lable_file))
            cutoff = min(len(currData), len(currLabel))
            currData = currData[:cutoff,:]
            currLabel =currLabel[:cutoff]
            
            
            if allFrames.shape[0] ==0:
                allFrames = currData
                allLables = currLabel
            else:
                allFrames = np.vstack((allFrames, currData))
                allLables = np.hstack((allLables, currLabel))

    return allFrames, allLables

def fix_data(X, y, shuff=False):
    
    c = Counter(y)
    print('Before:',X.shape, c)
    min_key, min_count = min(c.items(), key=itemgetter(1))
    print(min_key, min_count)
    if min_count<10:
        indx = np.where(y == min_key)
        dub = X[indx]
#         print(dub.shape)
        X = np.vstack((X, np.tile(dub, (10, 1))))
        y = np.hstack((y, np.repeat(min_key,len(dub)*10)))
        print('Dublicated :',X.shape, Counter(y))
    
        #     sm = resample(n_neighbors=max(min_count-1,1))
    sm = sv.MulticlassOversampling(resample())
    if shuff:
        X, y = shuffle(X, y)
        
    #     X, trainy = sm.fit_resample(X, y)
    X, y = sm.sample(X, y)
    print('After:',X.shape, Counter(y))
    
    return X, y

def load_dataset_selectedSubj(trainSubjs, valSubjs, testSubjs, label_folder, feature_folder,\
                              prblemType, featureType, num_classes):
    scaler = MinMaxScaler()
    
    #sampeling method: SMOTE, DeepSMOTE, DeepFake?
    # load all train
    trainX, trainy = load_sub_dataset(trainSubjs, label_folder, feature_folder, featureType)
    trainX = scaler.fit_transform(trainX)
    print('Train Data:')
    trainX, trainy = fix_data(trainX, trainy, shuff=True)


    # load validation
    valX, valy = load_sub_dataset(valSubjs, label_folder, feature_folder, featureType)
    valX = scaler.transform(valX)
    print('Val Data:')
    valX, valy = fix_data(valX, valy, shuff=True)
#     valX, valy = shuffle(valX, valy)
#     valX, valy = sm.fit_resample(valX, valy)
#     print(valX.shape, Counter(valy))
    # val_class_weight = sumary_data(valy)


    # load all test
    testX, testy = load_sub_dataset(testSubjs, label_folder, feature_folder, featureType)
    testX = scaler.transform(testX)
    print('Test Data:')
    testX, testy = fix_data(testX, testy, shuff=False)
#     testX, testy = sm.fit_resample(testX, testy)
#     print(testX.shape, Counter(testy))
    # test_class_weight = sumary_data(testy)

    # one hot encode y
    trainy = tf.keras.utils.to_categorical(trainy,  num_classes=num_classes)
    valy = tf.keras.utils.to_categorical(valy,  num_classes=num_classes)
    testy = tf.keras.utils.to_categorical(testy,  num_classes=num_classes)
    
    return trainX, trainy, valX, valy , testX, testy, num_classes
    
def create_modlling(label_folder,feature_folder,result_folder, prblemType, featureType, num_classes):    
    # repeat experiment
    temp = {}
    all_trainSubjs = [['F' + str(i) for i in [1, 2, 3, 4, 5, 6, 8]]]
    all_valSubjs = [['F' + str(i) for i in [11, 17]]]
    all_testSubjs = [['F' + str(i) for i in [7, 10, 13]]]
    
    min_budget = 9
    max_budget = 243
    n_iterations = 50
    num_workers = 12

    for r in range(len(all_trainSubjs)):
        shared_directory = result_folder + '_'+ str(r) 
        if os.path.exists(shared_directory):
            print(shared_directory,' already processed')
            continue
        
        print(shared_directory,' under processing')
        classType = os.path.basename(shared_directory)

        # load data
        trainSubjs = all_trainSubjs[r]
        valSubjs = all_valSubjs[r]
        testSubjs = all_testSubjs[r]
        
        trainX, trainy, valX, valy, testX, testy, num_classes = \
            load_dataset_selectedSubj(trainSubjs, valSubjs, testSubjs, \
                                      label_folder, feature_folder, prblemType, featureType, num_classes)
        
        
        n_timesteps, n_features, n_outputs = trainX.shape[0], trainX.shape[1], num_classes        
        
        host = hpns.nic_name_to_host('lo')
        result_logger = hpres.json_result_logger(directory=shared_directory, overwrite=True)
        NS = hpns.NameServer(run_id=classType, host=host, port=0, working_directory=shared_directory)
        ns_host, ns_port = NS.start()
    
        workers = []
        for i in range(num_workers):
            worker = KerasWorker(n_features, n_outputs, prblemType, \
                                 trainX, trainy, valX, valy, testX, testy, \
                                 shared_directory,
                                 run_id=classType,host=host, nameserver=ns_host, nameserver_port=ns_port,
                                 id=i)
            worker.run(background=True)
            workers.append(worker)

        bohb = BOHB(configspace=worker.get_configspace(),
                  run_id=classType,
                  host=host,
                  nameserver=ns_host,
                  nameserver_port=ns_port,
                  result_logger=result_logger,
                  min_budget=min_budget, max_budget=max_budget
                    )
        res = bohb.run(n_iterations=1,  min_n_workers=num_workers)

        id2config = res.get_id2config_mapping()
        incumbent = res.get_incumbent_id()

        print('Best found configuration:', id2config[incumbent]['config'])
        # print('A total of %i unique configurations where sampled.' % len(id2config.keys()))
        # print('A total of %i runs where executed.' % len(res.get_all_runs()))
        # print('Total budget corresponds to %.1f full function evaluations.' % (
        #             sum([r.budget for r in res.get_all_runs()]) / max_budget))

        # store results
        with open(os.path.join(shared_directory, 'results.pkl'), 'wb') as fh:
            pickle.dump(res, fh)

        # shutdown
        bohb.shutdown(shutdown_workers=True)
        NS.shutdown()


### modeling handcrafted 

In [4]:
prblemTypes = ['classification']
featureTypes = ['handcrafted']
classes = [2, 3,5,9]
# divides = [2.5, 5]
fusionTypes = ['2.5','5','5s']
# sampling_methods = ['kmeans_SMOTE','AMSCO','Gaussian_SMOTE','SOMO','NRAS','CCR','ANS']#['ADASYN']#'SMOTE','KMeansSMOTE',,'ADASYN','RUSBoostClassifier']

sampling_methods = ['distance_SMOTE','polynom_fit_SMOTE','ProWSyn']#,'SMOTE_ENN','CCR','ANS']#['ADASYN']#'SMOTE','KMeansSMOTE',,'ADASYN','RUSBoostClassifier']

permutations=[ prblemTypes, featureTypes, classes, fusionTypes]
all_permutations = list(itertools.product(*permutations))
print(len(all_permutations))
for this_permutation in all_permutations:
    (prblemType, featureType, eng_lvls, fusionType) = this_permutation
    classType = 'round_avg_eng_level' if prblemType == 'classification' else 'avg_eng_level'
#     classType = 'nana_eng_level'
    
    print('Working on: ',prblemType, featureType, eng_lvls, fusionType)
    
    label_folder = os.path.join(lables_path,'_'.join([classType,'eng_lvl',prblemType,str(eng_lvls),str(fusionType)]))
    feature_folder = os.path.join(features_path,'_'.join([featureType,'features',fusionType]))
    
    for sampling_method in sampling_methods:
#         from imblearn.over_sampling import exec(sampling_method) as resample
#         import importlib
#         resample = importlib.import_module(".".join(["imblearn.over_sampling"]), package=sampling_method)
#         resample = getattr(__import__("imblearn.over_sampling", globals(), locals(), [sampling_method], 0), sampling_method)
#         resample = my_import(".".join(["",]))
#         from imblearn.over_sampling import SMOTE
#         mod = __import__('imblearn.over_sampling', fromlist=[sampling_method])
#         resample = getattr(mod, sampling_method)
#         if sampling_method == 'SMOTEENN':
#             resample = getattr(__import__("imblearn.combine", globals(), locals(), [sampling_method], 0), sampling_method)
#         elif sampling_method == 'RUSBoostClassifier':
#             resample = getattr(__import__("imblearn.ensemble", globals(), locals(), [sampling_method], 0), sampling_method)
#         else:
#             resample = getattr(__import__("imblearn.over_sampling", globals(), locals(), [sampling_method], 0), sampling_method)
        
        resample = getattr(__import__("smote_variants", globals(), locals(), [sampling_method], 0), sampling_method)

        
#         print(resample)
#         sm = resample()
#     break

        #save results like: rgb_classification_9_2.5_none
        result_folder = os.path.join(results_path,'_'.join([featureType,prblemType,str(eng_lvls),fusionType,sampling_method]))

        create_modlling(label_folder,feature_folder,result_folder, prblemType, featureType, eng_lvls)#, divide,fusionType)


12
Working on:  classification handcrafted 2 2.5
./modelling_results/handcrafted_classification_2_2.5_distance_SMOTE_0  under processing
Train Data:
Before: (18236, 330) Counter({1.0: 14902, 0.0: 3334})
0.0 3334
After: (29804, 330) Counter({1.0: 14902, 0.0: 14902})
Val Data:
Before: (3467, 330) Counter({1.0: 3003, 0.0: 464})
0.0 464
After: (6006, 330) Counter({1.0: 3003, 0.0: 3003})
Test Data:
Before: (3916, 330) Counter({1.0: 3568, 0.0: 348})
0.0 348
After: (7136, 330) Counter({0.0: 3568, 1.0: 3568})


2021-11-08 08:56:47.373548: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-08 08:56:47.379895: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sharifa/catkin_ws/devel/lib:/usr/local/cuda-11.0/lib64
2021-11-08 08:56:47.380843: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sharifa/catkin_ws/devel/lib:/usr/local/cuda-11.0/lib64
2021-11-08 08:56:47.380859: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned ab

Best found configuration: {'dropout1_rate': 0.4883112688205267, 'dropout2_rate': 0.7030779683734437, 'lr': 6.768199141884079e-05, 'num_dense_layers': 1, 'optimizer': 'Adam', 'start_neurons_units': 403}
./modelling_results/handcrafted_classification_2_2.5_polynom_fit_SMOTE_0  under processing
Train Data:
Before: (18236, 330) Counter({1.0: 14902, 0.0: 3334})
0.0 3334
After: (28238, 330) Counter({1.0: 14902, 0.0: 13336})
Val Data:
Before: (3467, 330) Counter({1.0: 3003, 0.0: 464})
0.0 464
After: (5787, 330) Counter({1.0: 3003, 0.0: 2784})
Test Data:
Before: (3916, 330) Counter({1.0: 3568, 0.0: 348})
0.0 348
After: (7048, 330) Counter({1.0: 3568, 0.0: 3480})
Best found configuration: {'dropout1_rate': 0.8699527371073118, 'dropout2_rate': 0.10881826861757975, 'lr': 0.00046346691015292264, 'num_dense_layers': 1, 'optimizer': 'Adam', 'start_neurons_units': 346}
./modelling_results/handcrafted_classification_2_2.5_ProWSyn_0  under processing
Train Data:
Before: (18236, 330) Counter({1.0: 14902

After: (22254, 330) Counter({1.0: 7418, -1.0: 7418, 0.0: 7418})
Val Data:
Before: (1725, 330) Counter({1.0: 1499, 0.0: 179, -1.0: 47})
-1.0 47
After: (4497, 330) Counter({1.0: 1499, 0.0: 1499, -1.0: 1499})
Test Data:
Before: (1958, 330) Counter({1.0: 1785, 0.0: 172, -1.0: 1})
-1.0 1
Dublicated : (1968, 330) Counter({1.0: 1785, 0.0: 172, -1.0: 11})
After: (5355, 330) Counter({0.0: 1785, 1.0: 1785, -1.0: 1785})
Best found configuration: {'dropout1_rate': 0.03753864993743229, 'dropout2_rate': 0.5379529782686135, 'lr': 1.9614788905781026e-05, 'num_dense_layers': 3, 'optimizer': 'Adam', 'start_neurons_units': 89, 'dense1_units': 10, 'dense2_units': 49}
./modelling_results/handcrafted_classification_3_5_polynom_fit_SMOTE_0  under processing
Train Data:
Before: (9106, 330) Counter({1.0: 7418, 0.0: 1516, -1.0: 172})
-1.0 172
After: (22566, 330) Counter({0.0: 7580, -1.0: 7568, 1.0: 7418})
Val Data:
Before: (1725, 330) Counter({1.0: 1499, 0.0: 179, -1.0: 47})
-1.0 47
After: (4388, 330) Counter({

After: (3399, 330) Counter({0.0: 1133, 1.0: 1133, 2.0: 1133})
Best found configuration: {'dropout1_rate': 0.12215538423287063, 'dropout2_rate': 0.7854553584509553, 'lr': 0.001489413498844762, 'num_dense_layers': 2, 'optimizer': 'SGD', 'start_neurons_units': 40, 'dense1_units': 88, 'sgd_momentum': 0.7394059180642486}
./modelling_results/handcrafted_classification_5_5_polynom_fit_SMOTE_0  under processing
Train Data:
Before: (9106, 330) Counter({1.0: 5910, 2.0: 1516, 0.0: 1507, -1.0: 173})
-1.0 173
After: (24057, 330) Counter({2.0: 6064, -1.0: 6055, 0.0: 6028, 1.0: 5910})
Val Data:
Before: (1725, 330) Counter({1.0: 1158, 2.0: 341, 0.0: 179, -1.0: 47})
-1.0 47
After: (4336, 330) Counter({1.0: 1158, -1.0: 1081, 0.0: 1074, 2.0: 1023})
Test Data:
Before: (1958, 330) Counter({1.0: 1133, 2.0: 654, 0.0: 171})
0.0 171
After: (3638, 330) Counter({2.0: 1308, 0.0: 1197, 1.0: 1133})
Best found configuration: {'dropout1_rate': 0.5144860705088373, 'dropout2_rate': 0.6731229161069707, 'lr': 0.000495068

After: (11538, 330) Counter({0.0: 1923, 1.0: 1923, 2.0: 1923, 3.0: 1923, 4.0: 1923, -1.0: 1923})
Best found configuration: {'dropout1_rate': 0.48469966949217813, 'dropout2_rate': 0.5185390497887108, 'lr': 0.0002236786850124063, 'num_dense_layers': 2, 'optimizer': 'SGD', 'start_neurons_units': 129, 'dense1_units': 110, 'sgd_momentum': 0.7724646898366381}
Working on:  classification handcrafted 9 5
./modelling_results/handcrafted_classification_9_5_distance_SMOTE_0  under processing
Train Data:
Before: (9106, 330) Counter({2.0: 4548, 0.0: 1445, 1.0: 1410, 3.0: 1119, 4.0: 400, -2.0: 121, -1.0: 63})
-1.0 63
After: (31836, 330) Counter({2.0: 4548, 4.0: 4548, 0.0: 4548, 3.0: 4548, 1.0: 4548, -1.0: 4548, -2.0: 4548})
Val Data:
Before: (1725, 330) Counter({2.0: 1066, 3.0: 309, 0.0: 169, 1.0: 105, -2.0: 31, 4.0: 28, -1.0: 17})
-1.0 17
After: (7462, 330) Counter({2.0: 1066, 3.0: 1066, 0.0: 1066, 1.0: 1066, -2.0: 1066, 4.0: 1066, -1.0: 1066})
Test Data:
Before: (1958, 330) Counter({2.0: 947, 3.0: